## **Train a Variational Quantum Classifier (VQC)**

In [16]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from qiskit_aer import Aer
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGatesDecomposition, CommutativeCancellation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

In [17]:
# Step 1: Generate and Preprocess Data
X, y = make_classification(n_samples=100,
                          n_features=2, # Total number of features
                          n_informative=2, # Ensure sum <_ total of features
                          n_redundant=0, # No redundant features
                          n_clusters_per_class=1,
                          random_state=42)

In [18]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [19]:
# Split into training and testing sets
X_train,X_test,y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [20]:
# Step 2: Define Quantum Feature Map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')

In [21]:
# Step 3: Define Variational Quantum Circuit (Ansatz)
ansatz = RealAmplitudes(num_qubits=2, reps=3, entanglement='linear')

In [22]:
# Step 4: Initialize the StatevectorSampler (Qiskit 1.2+)
sampler = StatevectorSampler()

In [23]:
# Step 5: Define Fidelity Quantum Kernel
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=sampler)

In [24]:
# Step 6: Instantiate the Optimizer (Corrected Import)
optimizer =COBYLA(maxiter=100) # Ensure optimizer is an instance, not a string


In [25]:
# Step 7: Define a Custom pass Manager for Transpilation ( Fixes warnings )
pass_manager = PassManager([
    Optimize1qGatesDecomposition(basis=['u3', 'cx']),
    CommutativeCancellation()
])

In [26]:
# Step 8: Define the Variational Quantum Classifier (VQC)
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer= optimizer,
    sampler=sampler,
    pass_manager=pass_manager # Provide pass manager to avoid transpilation warnings
)

In [27]:
# Step 9: Train the VQC Model
vqc.fit(X_train, y_train)

# step 10: Evaluate the model
y_pred = vqc.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)

In [ ]:
print(accuracy)

0.75
